In [45]:
import os
import time
import math
import cv2

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as progress
from sklearn.decomposition.pca import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.pipeline import Pipeline

In [46]:
%%time
train_dir = '../Netology/Data/Whale/train'
test_dir = '../Netology/Data/Whale/test'

imgs_train = [cv2.imread(train_dir + '/' + file, cv2.IMREAD_GRAYSCALE) for file in os.listdir(train_dir)]

min_cols = 138
min_rows = 54


CPU times: user 19.2 s, sys: 288 ms, total: 19.4 s
Wall time: 19.4 s


In [12]:
plt.imshow(compressed_train_imgs[8000], cmap='gray')

NameError: name 'compressed_train_imgs' is not defined

In [ ]:
plt.imshow(cv2.resize(imgs_train[99],(130,75)), cmap='gray')

In [47]:
good_pics = [i for i in range(len(imgs_train)) if (imgs_train[i].shape[0]>=min_rows)and(imgs_train[i].shape[1]>=min_cols)]
imgs_train = [imgs_train[i] for i in good_pics]

In [48]:
compressed_train_imgs = [cv2.resize(img, (256, 256)) for img in imgs_train]
del imgs_train

In [49]:
filenames = [file for file in os.listdir(train_dir)]
filenames = [filenames[i] for i in good_pics]
filenames_test = [file for file in os.listdir(test_dir)]
X_train = [img.ravel() for img in compressed_train_imgs]
df = pd.DataFrame(data = filenames, columns = ['FileName'])
data2 = pd.read_csv('../Netology/Data/Whale/train.csv',names = ['FileName','WhaleID'])
data = data2.merge(df, on = 'FileName')
data = data.drop('FileName',axis = 1)

In [ ]:
"""#Extract filenames to later associate to whale IDs
filenames = [file for file in os.listdir(train_dir)]
filenames = [filenames[i] for i in good_pics]
filenames_test = [file for file in os.listdir(test_dir)]
#Convert images into pandas dataframe format for learning the model
df1 = pd.DataFrame(data = [img.ravel() for img in compressed_train_imgs])
df2 = pd.DataFrame(data = filenames, columns = ['FileName'])
data1 = pd.concat([df2,df1],axis = 1)
#Obtain filenames, indexed by whale IDs
data2 = pd.read_csv('./train.csv',names = ['FileName','WhaleID'])
#Map whale IDs to images via filenames (index of data1 is matched to value of data2)
data = data2.merge(data1, on = 'FileName')
data = data.drop('FileName',axis = 1)
del compressed_train_imgs, df1, df2, data1, data2
"""

In [50]:
del compressed_train_imgs, df, data2

In [51]:
y_train = data['WhaleID']

In [52]:
X_train=np.asarray(X_train)

In [53]:
knn=KNeighborsClassifier()

In [54]:
pca = PCA(n_components = 100, whiten = True)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [62]:
logreg = LogisticRegression()

In [63]:
clf = Pipeline([('pca',pca),
                ('logreg',logreg),
                ])


In [ ]:
clf.fit(X_train,y_train)

In [58]:
imgs_test = [cv2.imread(test_dir + '/' + file, cv2.IMREAD_GRAYSCALE) for file in os.listdir(test_dir)]
compressed_test_imgs = [cv2.resize(img, (256, 256)) for img in imgs_test]
del imgs_test

X_test = [img.ravel() for img in compressed_test_imgs]

del compressed_test_imgs

In [59]:
len(X_test)

15610

In [60]:
y_preds = clf.predict_proba(X_test)

In [61]:

results = pd.DataFrame(data = [clf.classes_[np.argsort(y_preds[i,:])[-5:]] for i in range(y_preds.shape[0])],
                       index = filenames_test)
def list_to_str(L):
    string = ""
    for word in L:
        string = string + word + " "
    return(string)
        
results2 = pd.DataFrame(data = [list_to_str(results.iloc[i].values) for i in range(results.shape[0])],
                        index = filenames_test,columns = ['Id'])
 
results2.to_csv('submission.csv', sep = ',', index_label = 'Image', header = True)
